In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (8), incompatible header (6))


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00221766s, CPU 0.002218s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00159266s

┌ Warning: Number of threads(1) ≠ logical threads in CPU(4).
└ @ LowLevelFEM ~/Dokumentumok/GitHub/LowLevelFEM.jl/src/general.jl:104


In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00065628s, CPU 0.000655s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0111221s, CPU 0.010991s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneStress)

Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98)

In [4]:
supp = FEM.displacementConstraint("left", ux=0, uy=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [5]:
u0 = FEM.initialDisplacement(problem, "body", ux=0, uy=0)

VectorField

196×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
FEM.initialDisplacement!("left", u0, ux=0, uy=0)
u0

VectorField

196×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
v0 = FEM.initialVelocity(problem, "body", vx=0, vy=0)

VectorField

196×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [8]:
FEM.initialVelocity!("body", v0, vx=0, vy=0)

In [9]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])

VectorField

196×1 Matrix{Float64}:
 0.0
 0.0
 0.062499999999999924
 0.0
 0.06250000000000028
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [10]:
FEM.applyBoundaryConditions!(K, M, f, [supp])

In [11]:
Tmin = FEM.smallestPeriodTime(K, M)

4.7358837432307986e-8

In [12]:
u, v = FEM.CDM(K, M, f, u0, v0, 10Tmin, Tmin / π)

VectorField

196×32 Matrix{Float64}:
 0.0  0.0         0.0          0.0          …   0.0          0.0
 0.0  0.0         0.0          0.0              0.0          0.0
 0.0  2.3727e-7   7.2407e-7    1.16273e-6       7.41914e-6   7.07078e-6
 0.0  0.0         1.02783e-7   2.80022e-7       2.51479e-7   2.5164e-7
 0.0  2.3727e-7   7.24291e-7   1.15655e-6       7.38629e-6   7.02337e-6
 0.0  0.0        -1.00405e-7  -2.79702e-7   …  -2.69542e-7  -2.30219e-7
 0.0  0.0         0.0          0.0              0.0          0.0
 0.0  0.0         0.0          0.0              0.0          0.0
 0.0  0.0         0.0          0.0              1.42918e-6   1.52186e-6
 0.0  0.0         0.0          0.0              9.03249e-7   9.22041e-7
 0.0  0.0         0.0          0.0          …   2.6716e-6    2.78816e-6
 0.0  0.0         0.0          0.0              1.10774e-6   1.14428e-6
 0.0  0.0         0.0          0.0              3.91715e-6   3.95238e-6
 ⋮                                          ⋱   ⋮           
 0.0  0.

VectorField

196×32 Matrix{Float64}:
 0.0   0.0      0.0        0.0        …    0.0         0.0         0.0
 0.0   0.0      0.0        0.0             0.0         0.0         0.0
 0.0  15.7395  32.2923    29.0989        -37.3475    -33.4644    -23.1091
 0.0   0.0      6.81822   11.7573         -2.84084     4.9213      0.0106678
 0.0  15.7395  32.307     28.6745        -40.2951    -38.3665    -24.0747
 0.0   0.0     -6.66045  -11.8938     …   -0.696088   -0.657404    2.60849
 0.0   0.0      0.0        0.0             0.0         0.0         0.0
 0.0   0.0      0.0        0.0             0.0         0.0         0.0
 0.0   0.0      0.0        0.0            -1.50707     2.17062     6.14843
 0.0   0.0      0.0        0.0             2.91829     3.68621     1.24659
 0.0   0.0      0.0        0.0        …    3.69823     5.66588     7.73208
 0.0   0.0      0.0        0.0            -2.56016    -0.680695    2.4238
 0.0   0.0      0.0        0.0            10.7891     10.3919      2.33718
 ⋮                

(, )

In [13]:
FEM.showDoFResults(u, :uvec, visible=true)

1

In [14]:
FEM.showDoFResults(v, :vvec)

2

In [15]:
e1 = FEM.solveStrain(u)

TensorField

162-element Vector{Matrix{Float64}}:
 [0.0 0.0 … 7.926674240248463e-6 7.507995471758838e-6; 0.0 0.0 … 4.5973060285585465e-7 3.1283932486156986e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.9759390330378963e-6 -2.0020645541125595e-6]
 [0.0 0.0 … 7.926674240248463e-6 7.507995471758838e-6; 0.0 0.0 … 4.5973060285585465e-7 3.1283932486156986e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.9759390330378963e-6 -2.0020645541125595e-6]
 [0.0 0.0 … 7.926674240248463e-6 7.507995471758838e-6; 0.0 0.0 … 4.5973060285585465e-7 3.1283932486156986e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.9759390330378963e-6 -2.0020645541125595e-6]
 [0.0 0.0 … 7.926674240248463e-6 7.507995471758838e-6; 0.0 0.0 … 4.5973060285585465e-7 3.1283932486156986e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.9759390330378963e-6 -2.0020645541125595e-6]
 [0.0 0.0 … 7.926674240248463e-6 7.507995471758838e-6; 0.0 0.0 … 4.5973060285585465e-7 3.1283932486156986e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.9759390330378963e-6 -2.0020645541125595e-6]
 [0

In [16]:
FEM.showStrainResults(e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [17]:
e2 = FEM.solveStrain(u, DoFResults=true)

TensorField

882×32 Matrix{Float64}:
 0.0   0.0          0.0           0.0         …   1.06814e-5   1.12088e-5
 0.0   0.0          0.0           0.0             5.33977e-6   5.18097e-6
 0.0   0.0          0.0           0.0             0.0          0.0
 0.0   0.0          0.0           0.0             5.33977e-6   5.18097e-6
 0.0   0.0          0.0           0.0            -1.13423e-6  -1.22925e-6
 0.0   0.0          0.0           0.0         …   0.0          0.0
 0.0   0.0          0.0           0.0             0.0          0.0
 0.0   0.0          0.0           0.0             0.0          0.0
 0.0   0.0          0.0           0.0            -4.09165e-6  -4.27695e-6
 0.0   2.58514e-6   5.49481e-6    5.16662e-6      3.86493e-6   4.40506e-6
 0.0  -7.78935e-9   5.83199e-7    8.05069e-7  …   1.37276e-6   5.40184e-7
 0.0   0.0          0.0           0.0             0.0          0.0
 0.0  -7.78935e-9   5.83199e-7    8.05069e-7      1.37276e-6   5.40184e-7
 ⋮                                            ⋱  

In [18]:
FEM.showDoFResults(e2, :e)

4

In [19]:
e3 = FEM.elementsToNodes(e1)

TensorField

882×32 Matrix{Float64}:
 0.0  0.0  0.0   9.08482e-7   3.54476e-6  …   7.92667e-6   7.508e-6
 0.0  0.0  0.0  -1.5316e-7   -4.62263e-7      4.59731e-7   3.12839e-7
 0.0  0.0  0.0   0.0          0.0             0.0          0.0
 0.0  0.0  0.0  -1.5316e-7   -4.62263e-7      4.59731e-7   3.12839e-7
 0.0  0.0  0.0  -4.282e-8    -4.35893e-8     -3.31615e-6  -2.83651e-6
 0.0  0.0  0.0   0.0          0.0         …   0.0          0.0
 0.0  0.0  0.0   0.0          0.0             0.0          0.0
 0.0  0.0  0.0   0.0          0.0             0.0          0.0
 0.0  0.0  0.0  -3.70998e-7  -1.5005e-6      -1.97594e-6  -2.00206e-6
 0.0  0.0  0.0   9.08482e-7   3.54476e-6      7.92667e-6   7.508e-6
 0.0  0.0  0.0  -1.5316e-7   -4.62263e-7  …   4.59731e-7   3.12839e-7
 0.0  0.0  0.0   0.0          0.0             0.0          0.0
 0.0  0.0  0.0  -1.5316e-7   -4.62263e-7      4.59731e-7   3.12839e-7
 ⋮                                        ⋱   ⋮           
 0.0  0.0  0.0   0.0          0.0         …   

In [20]:
FEM.showDoFResults(e3, :e)

5

In [21]:
s1 = FEM.solveStress(u)

TensorField

162-element Vector{Matrix{Float64}}:
 [0.0 0.0 … 1.5234789651209784 1.463086157826369; 0.0 0.0 … 0.07072778505474688 0.04812912690177998; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 1.5234789651209784 1.463086157826369; 0.0 0.0 … 0.07072778505474688 0.04812912690177998; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 1.5234789651209784 1.463086157826369; 0.0 0.0 … 0.07072778505474688 0.04812912690177998; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 1.5234789651209784 1.463086157826369; 0.0 0.0 … 0.07072778505474688 0.04812912690177998; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 1.5234789651209784 1.463086157826369; 0.0 0.0 … 0.07072778505474688 0.04812912690177998; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 1.5234789651209784 1.463086157826369; 0.0 0.0 … 0.07072778505474688 0.04812912690177998; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 1.5234789651209784 1.463086157826369; 0.0 0.0 … 0.07072778505474688 0.04812912690177998; … ; 

In [22]:
FEM.showStressResults(s1, :s)

6

In [23]:
s2 = FEM.solveStress(u, DoFResults=true)

TensorField

882×32 Matrix{Float64}:
 0.0   0.0          0.0         …   2.25119      2.27278     2.38242
 0.0   0.0          0.0             0.398054     0.410751    0.398536
 0.0   0.0          0.0             0.0          0.0         0.0
 0.0   0.0          0.0             0.398054     0.410751    0.398536
 0.0   0.0          0.0             0.481635     0.454988    0.468877
 0.0   0.0          0.0         …   0.0          0.0         0.0
 0.0   0.0          0.0             0.0          0.0         0.0
 0.0   0.0          0.0             0.0          0.0         0.0
 0.0   0.0          0.0             0.0          0.0         0.0
 0.0   0.568164     1.16997         0.985032     0.761258    0.904988
 0.0  -0.00059918   0.0448615   …  -0.0068722    0.105597    0.0415526
 0.0   0.0          0.0             0.0          0.0         0.0
 0.0  -0.00059918   0.0448615      -0.0068722    0.105597    0.0415526
 ⋮                              ⋱                ⋮          
 0.0   0.0          0.0         … 

In [24]:
FEM.showDoFResults(s2, :s)

7

In [25]:
err1 = FEM.fieldError(e1)

TensorField

882×32 Matrix{Float64}:
 0.0  0.0  0.0  0.0          …  0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0          …  0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0          …  0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 0.0  0.0  0.0  0.0             0.0          0.0          0.0  0.0
 ⋮                           ⋱                            ⋮    
 0.0  0.0  0.0  0.0          …  0.0      

In [26]:
FEM.showDoFResults(err1, :tensor)

8

In [27]:
u2, v2 = FEM.HHT(K, M, f, u0, v0, 10Tmin, Tmin / π)

VectorField

196×32 Matrix{Float64}:
 0.0   0.0           0.0           0.0          …   0.0          0.0
 0.0   0.0           0.0           0.0              0.0          0.0
 0.0   1.96992e-7    6.70457e-7    1.18801e-6       7.88745e-6   7.24592e-6
 0.0   1.3694e-8     8.23964e-8    2.283e-7         1.86508e-7   1.96365e-7
 0.0   1.96877e-7    6.69414e-7    1.18391e-6       7.84996e-6   7.33688e-6
 0.0  -1.34752e-8   -8.12621e-8   -2.2574e-7    …  -1.15827e-7  -1.6459e-7
 0.0   0.0           0.0           0.0              0.0          0.0
 0.0   0.0           0.0           0.0              0.0          0.0
 0.0   5.46128e-15   1.51574e-13   2.0631e-12       1.4767e-6    1.55798e-6
 0.0   9.06609e-16   2.67301e-14   3.85436e-13      8.26856e-7   8.29535e-7
 0.0   6.28236e-14   1.53431e-12   1.83474e-11  …   2.54485e-6   2.60218e-6
 0.0   1.06303e-14   2.75197e-13   3.47841e-12      1.19396e-6   1.17208e-6
 0.0   7.47485e-13   1.56826e-11   1.60715e-10      3.80449e-6   3.76908e-6
 ⋮               

VectorField

196×32 Matrix{Float64}:
 0.0   0.0          0.0          …    0.0        0.0          0.0
 0.0   0.0          0.0               0.0        0.0          0.0
 0.0  26.1353      36.6801          -25.9887   -40.8797     -44.2329
 0.0   1.81681      7.29807          -4.32503   -1.99445      3.30228
 0.0  26.1201      36.5722          -25.6902   -31.245      -36.8259
 0.0  -1.78778     -7.20563      …    6.58993    0.400518    -6.87004
 0.0   0.0          0.0               0.0        0.0          0.0
 0.0   0.0          0.0               0.0        0.0          0.0
 0.0   7.24558e-7   1.86605e-5        6.4047     7.4016       3.3813
 0.0   1.20281e-7   3.30577e-6       -1.06085   -0.38648      0.741819
 0.0   8.33493e-6   0.00018689   …   -2.48894    1.40766      6.19952
 0.0   1.41034e-6   3.36903e-5        3.20235   -0.0655877   -2.83676
 0.0   9.91702e-5   0.0018823         1.93328   -1.70819     -2.99027
 ⋮                               ⋱               ⋮          
 0.0   0.137826     1.1

(, )

In [28]:
FEM.showDoFResults(u2, :uvec)

9

In [29]:
FEM.showDoFResults(v2, :vvec)

10

In [30]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [31]:
gmsh.finalize()